# AI Training
## Edge model training

In [3]:
!python3 -m pip install tensorflow
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

  Using cached tensorflow-2.18.0-cp39-cp39-macosx_12_0_arm64.whl (239.4 MB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl (71 kB)
     |████████████████████████████████| 2.9 MB 709 kB/s eta 0:00:01
  Using cached numpy-2.0.2-cp39-cp39-macosx_14_0_arm64.whl (5.3 MB)
  Using cached wrapt-1.17.2-cp39-cp39-macosx_11_0_arm64.whl (38 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl (30 kB)
  Using cached tensorflow_io_gcs_filesystem-0.37.1-cp39-cp39-macosx_12_0_arm64.whl (3.5 MB)
     |████████████████████████████████| 1.3 MB 2.1 MB/s eta 0:00:01
  Using cached packaging-24.2-py3-none-any.whl (65 kB)
  Using cached protobuf-5.29.3-cp38-abi3-macosx_10_9_universal2.whl (417 kB)
  Using cached ml_dtypes-0.4.1-cp39-cp39-macosx_10_9_universal2.w

In [7]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Input, Model, layers
from sklearn.model_selection import train_test_split

# Load and preprocess data
RAIN_THRESHHOLD = 0.5
data = pd.read_csv('weather_data.csv', low_memory=False)
data = data.sample(frac=1, random_state=2).reset_index(drop=True)
data = data.replace('', float('nan')).dropna()

# Encode rain to 1 or 0 depending on threshold
y = data['rain'].apply(lambda x: 1 if x > RAIN_THRESHHOLD else 0)
X = data.drop(columns=['rain', 'date'])

# Encode categorical columns
for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

# Ensure all data is numeric
X = X.astype(float)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape y to (num_samples, 1) for regression
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

# Define the model
inputs = Input(shape=(X_train.shape[1],))
x = layers.Dense(64, activation='relu', name='e1')(inputs)
x_edge = layers.Dense(32, activation='relu', name='e2')(x)
edge_outputs = layers.Dense(1, activation='linear', name='edge_output')(x_edge)
x = layers.Dense(64, activation='relu', name='controlmodel')(x_edge)
x = layers.Dense(128, activation='relu', name='l1')(x)
x = layers.Dense(256, activation='relu', name='l2')(x)
x = layers.Dense(128, activation='relu', name='l3')(x)
x = layers.Dense(64, activation='relu', name='l4')(x)
control_outputs = layers.Dense(1, activation='linear', name='control_output')(x)

# Create and compile the combined model
combined_model = Model(inputs=inputs, outputs=[edge_outputs, control_outputs])
combined_model.compile(optimizer='adam',
                       loss={'edge_output': 'mean_squared_error',
                             'control_output': 'mean_squared_error'},
                       metrics={'edge_output': 'mae',
                                'control_output': 'mae'})

# Train the combined model
combined_model.fit(X_train, [y_train, y_train], 
                   epochs=10, batch_size=32, 
                   validation_data=(X_test, [y_test, y_test]))

# Define the edge_model
edge_model = Model(inputs=inputs, outputs=[combined_model.get_layer('e2').output, edge_outputs])

# x§This needs training 
# Set last as untrainable, then train last ones, this is not needed for contorl model becasue its last output is trained when entire model 
# gets trained
for layer in edge_model.layers:
    layer.trainable = False

edge_model.get_layer('edge_output').trainable = True

edge_model.compile(optimizer='adam',
                   loss={'e2': None, 'edge_output': 'mean_squared_error'},  # Ignore loss for 'e2', we only care about the output of the inference
                   metrics={'edge_output': 'mae'})

edge_model.fit(X_train, [y_train, y_train], 
               epochs=10, batch_size=32, 
               validation_data=(X_test, [y_test, y_test]))

# Define the control_model
control_model_input = Input(shape=(32,))  # Shape of the intermediate layer output
z = combined_model.get_layer('controlmodel')(control_model_input)
z = combined_model.get_layer('l1')(z)
z = combined_model.get_layer('l2')(z)
z = combined_model.get_layer('l3')(z)
z = combined_model.get_layer('l4')(z)
control_model_outputs = combined_model.get_layer('control_output')(z)
control_model = Model(inputs=control_model_input, outputs=control_model_outputs)

# Inspect the models
edge_model.summary()
control_model.summary()


Epoch 1/10
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 8s 2ms/step - control_output_loss: 0.5303 - control_output_mae: 0.2954 - edge_output_loss: 145.1925 - edge_output_mae: 3.4171 - loss: 145.7229 - val_control_output_loss: 0.0668 - val_control_output_mae: 0.2123 - val_edge_output_loss: 0.1433 - val_edge_output_mae: 0.3114 - val_loss: 0.2101
Epoch 2/10
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - control_output_loss: 0.0536 - control_output_mae: 0.1401 - edge_output_loss: 0.1630 - edge_output_mae: 0.3001 - loss: 0.2166 - val_control_output_loss: 0.0508 - val_control_output_mae: 0.1058 - val_edge_output_loss: 0.5031 - val_edge_output_mae: 0.6292 - val_loss: 0.5539
Epoch 3/10
3184/3184 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - control_output_loss: 0.0471 - control_output_mae: 0.1063 - edge_output_loss: 0.3022 - edge_output_mae: 0.3900 - loss: 0.3494 - val_control_output_loss: 0.0454 - val_control_output_mae: 0.0882 - val_edge_output_loss: 4.7316 - val_edge_output_mae: 2.0258 - val_loss: 4.7771
Epoch 4/10
3

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 19)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e1 (Dense)                      │ (None, 64)             │         1,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ e2 (Dense)                      │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ edge_output (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,461 (13.52 KB)

 Trainable params: 33 (132.00 B)

...

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ controlmodel (Dense)            │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l1 (Dense)                      │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l2 (Dense)                      │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l3 (Dense)                      │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l4 (Dense)                      │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ control_output (Dense)          │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 84,673 (330.75 KB)

 Trainable params: 84,673 (330.75 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:

results = combined_model.evaluate(X_test, [y_test, y_test])
print("Test Loss:", results[0])
print("Edge Output - MAE:", results[1])
print("Edge Output - MSE - How much its off by on average:", results[2])
print("Control Output - MAE:", results[3])
print("Control Output - MSE:", results[4])

796/796 ━━━━━━━━━━━━━━━━━━━━ 0s 518us/step - control_output_loss: 0.0468 - control_output_mae: 0.0913 - edge_output_loss: 0.0480 - edge_output_mae: 0.1352 - loss: 0.0948
Test Loss: 0.09326030313968658
Edge Output - MAE: 0.04743298888206482
Edge Output - MSE - How much its off by on average: 0.04583030194044113
Control Output - MAE: 0.09009821712970734
Control Output - MSE: 0.13423798978328705


In [21]:
edge_model.export("expt")
converter = tf.lite.TFLiteConverter.from_saved_model('./expt')
tflite_model = converter.convert()
with open('../deployment/models/edge_model.tflite', 'wb') as f:
  f.write(tflite_model)

edge_model.save('../deployment/models/edge_model.keras')
    
control_model.save('../deployment/models/control_model.keras')


INFO:tensorflow:Assets written to: expt/assets


INFO:tensorflow:Assets written to: expt/assets


Saved artifact at 'expt'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 19), dtype=tf.float32, name='keras_tensor_54')
Output Type:
  List[TensorSpec(shape=(None, 32), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)]
Captures:
  4531610320: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4531613584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4531617424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4531613392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4531616272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  4531615888: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1741185165.733189   73505 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1741185165.735712   73505 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-05 14:32:45.738432: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: ./expt
2025-03-05 14:32:45.738945: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-05 14:32:45.738952: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: ./expt
2025-03-05 14:32:45.749383: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-05 14:32:45.825329: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: ./expt
2025-03-05 14:32:45.830744: I tensorflow/cc/saved_model/loader.cc:466] SavedModel load for tags { serve }; Status: success: OK. Took 92315 microseconds.


# On the AI Edge Server

In [8]:
from ai_edge_litert.interpreter import Interpreter
interpreter = Interpreter(model_path='../deployment/models/edge_model.tflite')
signatures = interpreter.get_signature_list()
interpreter.allocate_tensors()

# FEED IN THE INPUT DATA
input_details = interpreter.get_input_details()
input_data = np.array([[0,0,8.7,0,8.6,8.5,11.1,99,1003.1,2,12,2,350,61,66,0.0,15000,45,8]], dtype=np.float32)
# Set input tensor
input_index = input_details[0]['index']
interpreter.set_tensor(input_index, input_data)
# GET THE OUTPUT DATA
interpreter.invoke()
output_details = interpreter.get_output_details()
rain_data = interpreter.get_tensor(output_details[0]['index'])
node_data = interpreter.get_tensor(output_details[1]['index'])
node_data = node_data[0] # Send this to control
prob_rain = rain_data[0]


# Determine if it's raining
RAIN_THRESHOLD = 0.5
prob_rain
node_data

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


array([   0.      ,    0.      , 1736.833   ,    0.      ,    0.      ,
        122.79834 ,    0.      ,    0.      ,    0.      ,  498.57092 ,
        308.47418 ,    0.      ,    0.      ,    0.      , 2261.8982  ,
       1048.9276  ,    0.      ,    0.      ,  886.5067  ,    0.      ,
          0.      , 1980.9333  ,  846.6481  ,    0.      , 1223.5721  ,
          0.      ,   91.276794, 1690.1853  , 1646.5259  ,  399.7732  ,
          0.      ,  788.00134 ], dtype=float32)

# Control Model
## On control model

In [24]:
import numpy as np
import tensorflow as tf

## Vars that need to be defined for the function 
# Recieve node_data, data_result
BATCH_SIZE = 1 
X_data = []  
Y_data = []  
data_result = 0.4
data_result = np.array([data_result])

edge_model_result = node_data
edge_model_result = np.reshape(edge_model_result, (-1, 32)) 

control_predictions = control_model.predict(edge_model_result)
print("Control predictions:", control_predictions)

# IF data result exists, add it and intermediate layer to batches
if data_result.size > 0: 
    # Add data_result to Y and beginning nodes to X
    X_data.append(edge_model_result) 
    Y_data.append(data_result) 
    if len(X_data) >= BATCH_SIZE:
        numpy_X_data = np.vstack(X_data)
        numpy_Y_data = np.vstack(Y_data)  

        # Clear lists for the next batch
        X_data.clear()
        Y_data.clear()
        
        dataset = tf.data.Dataset.from_tensor_slices((numpy_X_data, numpy_Y_data))
        dataset = dataset.batch(BATCH_SIZE)
        # Compile and train the control_model
        control_model.compile(optimizer='adam',
                              loss='mean_squared_error',
                              metrics=['mae'])
        
        print("Training Model")
        # Below is how models are usually trained, we need to step down from the high level api of fit() and get slightly lower down in
        # tensorflow mush
        # history = control_model.fit(numpy_X_data, numpy_Y_data,
        #                             epochs=1, batch_size=BATCH_SIZE)
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

        for epoch in range(5):
            with tf.GradientTape() as tape: # This will track computations performed on tensors inside the block 
                for batch_X, batch_Y in dataset:
                    predictions = control_model(batch_X, training=True) # This is the forward pass 
                    loss = tf.keras.losses.MeanSquaredError()(batch_Y, predictions)
                gradients = tape.gradient(loss, control_model.trainable_variables) # gradients get calculated via calling tape.gradient 
                optimizer.apply_gradients(zip(gradients, control_model.trainable_variables))
            print(f"Epoch {epoch+1}, Loss: {loss.numpy()}")

print("done")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Control predictions: [[0.12519471]]
Training Model
Epoch 1, Loss: 0.07551795989274979
Epoch 2, Loss: 0.04582943767309189
Epoch 3, Loss: 0.01828954741358757
Epoch 4, Loss: 0.001427816110663116
done


2025-03-06 17:54:54.655685: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [7]:
!pip show tensorflow

Name: tensorflow
Version: 2.18.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, termcolor, typing-extensions, wrapt
Required-by: 


## Split Learning on Edge

In [6]:
edge_learning_model = tf.keras.models.load_model("../deployment/models/edge_model.keras")
edge_learning_model.compile(optimizer='adam',
                     loss='mean_squared_error',
                     metrics=['mae'])
# Extract gradients from the source model
source_gradients = tape.gradient(loss, source_model.trainable_variables)

# Step 4: Apply gradients to the target model
# Create an optimizer for the target model
optimizer = optimizers.SGD(learning_rate=0.01)

# Zip gradients with target model's trainable variables
gradient_target_pairs = zip(source_gradients, target_model.trainable_variables)

# Apply gradients to the target model
optimizer.apply_gradients(gradient_target_pairs)
